Imports

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as tfrms
from torchvision import models
from sklearn.model_selection import train_test_split
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm 
from sklearn.preprocessing import LabelEncoder
from torch.optim import Adam



Parameters

In [27]:
picture = "./train"
labelFile = "SignCSV.csv"
epochValue = 30
imageDimension = (32,32,3)
testRatio = 0.2
validationRatio = 0.2
batch_size_val = 100


Split Data

In [28]:
def loadData():
    # Read the CSV file to get the labels
    df = pd.read_csv(labelFile)
    
    # Lists to store images and their labels
    images = []
    labels = []
    
    # Iterate over each row in the CSV file
    for index, row in df.iterrows():
        # Get the image filename and label from the CSV
        image_file = row['filename']  # Change this column name if needed
        label = row['label']  # Change this column name if needed
        
        # Construct the full path to the image file
        image_path = os.path.join(picture, image_file)
        
        try:
            # Open and resize the image, then convert it to an array
            img = Image.open(image_path).convert("RGB")
            img = img.resize(32,32)
            img_array = np.array(img)
            
            # Append the image and label to the respective lists
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
    
    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels


    

Data Loader

In [32]:
# Define any transformations (e.g., normalization, resizing if not already done)
dTransforms = tfrms.Compose([
    tfrms.ToPILImage(),
    tfrms.ColorJitter(hue=0.4),
    tfrms.RandomHorizontalFlip(1),
    tfrms.RandomVerticalFlip(1),
    tfrms.RandomAffine(degrees = 15, shear=2),
    tfrms.CenterCrop(32),
    tfrms.Grayscale(num_output_channels=3),
    tfrms.ToTensor()
])


Convolutional Nueral Network

In [33]:
class DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=2,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=320,kernel_size=3,padding=1),
            nn.ELU(inplace=True),
            nn.Conv2d(in_channels=320,out_channels=256,kernel_size=3,padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.ELU(inplace=True),
        )

        self.classifcation = nn.Sequential(
            nn.Dropout(0,5),
            nn.Linear(16*256, 600),
            nn.ReLU(inplace=True),
            nn.Dropout(0,5),
            nn.Linear(in_features=600, out_features=256),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        X = self.conv(x)
        X = X.view(X.shape[0], -1)
        Y = self.classifcation(X)
        return Y

Training Function

In [53]:
def train(model, dataset, epochs=10, batch_size=32, learning_rate=0.001, device=None):

    # Use GPU if available, otherwise fall back to CPU
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)  # Move the model to the chosen device

    # Set up DataLoader for batching and shuffling
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    # Set up the optimizer (Adam) and loss function (CrossEntropyLoss)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        # Iterate over the DataLoader to get batches of images and labels
        for images, labels in tqdm(data_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            # Move images and labels to the same device as the model
            images = images.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass: Get model outputs
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass: Compute gradients
            loss.backward()

            # Optimize the model parameters
            optimizer.step()

            # Track the loss for reporting
            running_loss += loss.item()

            # Track accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        # Print the epoch results
        avg_loss = running_loss / len(data_loader)
        accuracy = correct_predictions / total_samples * 100

        print(f"Epoch {epoch+1}/{epochs}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.2f}%")

    return model

Load Data

In [ ]:
# Dataset Class to load images and labels
class NewDataset(torch.utils.data.Dataset):
    def __init__(self, images_folder, labels_csv, image_size=(32, 32), transform=None):
        self.images_folder = images_folder
        self.labels_df = pd.read_csv(labels_csv)
        self.image_size = image_size
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        image_name = self.labels_df.iloc[idx, 0]  # assuming first column is image filename
        label = self.labels_df.iloc[idx, -1]  # assuming last column is the label
        
        # Construct the full image path
        image_path = os.path.join(self.images_folder, str(image_name))  # Ensure correct path
        
        # Check if the image exists
        if not os.path.exists(image_path):
            print(f"Warning: Image not found: {image_path}")
            # Return a default image (e.g., a blank image) or skip the sample
            return None

        try:
            # Open and process the image
            image = Image.open(image_path).convert('RGB')
            image = image.resize(self.image_size)
            
            if self.transform:
                image = self.transform(image)
            
            return image, label
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None  # In case of error, return None

# Data loading and preprocessing
def encode_labels(dataset):
    if dataset.labels_df['label'].dtype == object:
        label_encoder = LabelEncoder()
        dataset.labels_df['encoded_label'] = label_encoder.fit_transform(dataset.labels_df['label'])
    else:
        label_encoder = None
    return dataset, label_encoder

# Example to load the data and train
def load_and_train(model, images_folder, labels_csv, batch_size, epochs, learning_rate, device):
    # Load dataset
    transform = tfrms.Compose([
        tfrms.ToTensor(),
        tfrms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Example normalization
    ])
    
    dataset = NewDataset(images_folder, labels_csv, image_size=(32, 32), transform=transform)
    
    # Initialize DataLoader
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # Optimizer and loss function
    optimizer = Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Set model to device
    model.to(device)

    # Start training loop
    for epoch in range(epochs):
        model.train()
        correct_predictions = 0
        total_samples = 0
        
        # Iterate over the DataLoader
        for images, labels in tqdm(data_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            # Skip None images and labels
            if images is None or labels is None:
                continue

            # Move data to device
            images, labels = images.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Track performance
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        # Calculate accuracy
        accuracy = (correct_predictions / total_samples) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

    return model

# Path to dataset (replace with actual paths)
images_folder = "./train"
labels_csv = "SignCSV.csv"

# Initialize the model
model = DCNN()

# Set the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
trained_model = load_and_train(model, images_folder, labels_csv, batch_size=32, epochs=10, learning_rate=0.001, device=device)

# The trained model is now ready to make predictions